In [1]:
import numpy as np

# Confidence Intervals

A common task is to use the result of our point estimator together with its mean square error to compute an interval in which we expect to often see the parameter of interest. Again we will assign a level of confidence to this statement in the form of a probability, and our interpretation of that is that it is a statement about the repeatability of the result. 

## Example

We are going to sample an exponential random variable $Y$ once and use that value to estimate the mean $\theta$ of the distribution.  We will then use the value of $Y$ to construct an interval in which we are 99% confident we will find $\theta$.

The PDF for the distribution is therefore:

$$ f(y) = \left\{ \begin{matrix} \frac{1}{\theta} e^{-y/\theta} & y \geq 0 \\ 0 & \mbox{otherwise} \end{matrix} \right. $$

The standard exponential (the exponential distribution with mean 1) is found from $Y$ by scaling it by $\theta$:  

$$ U = Y / \theta $$ 

will have the distribution 

$$ f_U(u) = \left\{ \begin{matrix} e^{-u} & u \geq 0 \\ 0 & \mbox{otherwise} \end{matrix} \right. $$

Which we need because this is the exponential distribution encoded in scipy.stats.expon.

We need to find two numbers $a$ and $b$ such that 

$$ P(a < U < b) = 0.99 $$

Usually we would do this so that the tails are symmetric:

$$ P( U < a) = 0.005 \quad \mbox{and} \quad P(U > b) = 0.005 $$

However it is worth noting that there are cases where we might want to have this be asymmetric such as by taking a=0 and choosing the b tail to have size 0.01.

In [2]:
# let's bring in the relevant distribution

from scipy.stats import expon

In [11]:
# We can find a by using the ppf, the inverse cdf:

a = expon.ppf(0.05)
a

0.051293294387550536

In [4]:
# Check with the CDF that this is the left tail at size 0.005

expon.cdf(a)

0.005

In [12]:
# To find b we need to take 1 - 0.005 as our inverse for the cdf:

b = expon.ppf(1-0.05)
b

2.99573227355399

In [13]:
# Check that the density between a and b is the 0.99 we want

expon.cdf(b) - expon.cdf(a)

0.8999999999999999

Note why this example is not symmetric:  We are using a distribution that is skewed left and only supported from $[0, \infty)$.

Okay so we have that:

$$ 0.99 = P(a < \frac{Y}{\theta} < b ) $$ 

and therefore solving for $\theta$ we get:

$$ 0.99 = P( \frac{Y}{b} < \theta < \frac{Y}{a} ) $$

Noting that we had to flip the inequalities when we took the reciprical. 

#### Suppose $Y= 5$

Then our conclusion is that there is a 99% probability that the mean of the exponential distribution we just sampled is in the interval:

In [14]:
(5/b, 5/a)

(1.669041003476671, 97.47862873111843)

#### Why so big?

This interval is huge!  Why is it so big?

Again we see these competing tensions:  adjusting our confidence will adjust the size of the region we end up with; and sampling our variable enough times will change the distribution for our point estimate.

## Using a Point Estimate from a Sample

Consider the following sample from the exponential distribution with mean $\theta$.  Let's pretend we do not know what $\theta$ is:

In [35]:
n = 100
sample = expon.rvs(size=n)*10
sample

array([2.53165791e+00, 6.24243106e+00, 1.41058851e+01, 2.55152947e-02,
       2.71135619e+01, 2.37936357e+00, 4.46821686e+00, 8.46233799e-01,
       9.68911235e+00, 1.30426098e+01, 5.72087706e+00, 1.99166961e+00,
       1.31122074e+01, 1.30085491e+00, 2.12898546e+01, 3.23896028e-02,
       2.28133528e+00, 1.27346246e+01, 2.44292451e+00, 1.75988893e+00,
       3.54919987e+00, 9.14209144e+00, 1.22420811e+00, 1.38443138e+01,
       2.48462044e+00, 7.09082179e+00, 8.32469722e+00, 3.88995880e+00,
       4.20824181e+00, 1.76244506e+01, 5.39770261e+00, 9.36255521e+00,
       8.81429884e+00, 1.68795819e+01, 2.20595834e+00, 4.94385182e+00,
       2.05088677e+00, 1.19313888e+01, 2.06459485e+01, 1.19474522e+01,
       3.61343451e+01, 8.74920010e+00, 2.87060926e+01, 7.94031682e+00,
       1.82516462e+01, 2.44890231e+01, 8.08982021e+00, 1.77964719e+01,
       1.77172555e+01, 3.56362118e+00, 9.76260383e-01, 6.01298475e+00,
       1.63808915e+00, 1.89666715e+00, 1.87140809e+00, 5.29025638e+00,
      

In [36]:
# Our point estimate for the distribution mean would be the sample mean:

Ybar = np.mean(sample)
Ybar

9.683165414180051

Which we expect to be normally distributed with mean $\theta$ and variance $\sigma^2 / \sqrt{n}$.  The problem is that we do not know $\sigma^2$.  We can use the sample variance $s^2$ instead, but it means rather than a normal distribution we expect that 

$$ T = \sqrt{n} \frac{\bar{Y} - \theta}{s} $$

will satisfy the Student's T distribution with $n-1$ degrees of freedom.

In [37]:
# Let me introduce you to the numpy.std function for finding standard deviations;
# you need to pass it a ddof of 1 to get the sample standard deviation.

s = np.std(sample, ddof=1)
s

8.624414901337017

In [38]:
# The next step is to find the interval

# import the t distribution

from scipy.stats import t

In [39]:
# we will use symmetric tails with density 0.005

a = -t.ppf(0.005, n-1)
a

2.626405456385186

In [40]:
# check that our interval has the correct density

t.cdf(a, n-1) - t.cdf(-a, n-1)

0.9899999999754332

So what we now have is that:

$$ 0.99 = P(-a < \sqrt{n} \frac{\bar{Y} - \theta}{s} < a ) $$

Solving for $\theta$ we get:

$$ \bar{Y} - \frac{a s}{\sqrt{n}} < \theta < \bar{Y} + \frac{a s}{\sqrt{n}} $$

Or in other words $\theta$ is in the following interval with 99% probability:

In [41]:
(Ybar - a*s/np.sqrt(n), Ybar + a*s/np.sqrt(n) )

(7.418044378679926, 11.948286449680175)

## Discussion

Note that 99% confident is pretty confident. Redo the computaiton above but looking for an interval with only 90% confidence. 10% of the time we will have the wrong conclusion!

In [42]:
n = 20

for k in range(100):
    sample = expon.rvs(size=n)*10
    Ybar = np.mean(sample)
    s = np.std(sample, ddof=1)
    a = -t.ppf(0.05, n-1)
    
    l = Ybar - a*s/np.sqrt(n)
    u = Ybar + s*s/np.sqrt(n)
    
    # check if 10 is in the interval and print in red; else print in black  
    if l > 10 or u < 10:
        print('\033[91m' + str((l, u )) + '\033[0m' )
    else:
        print((l, u))

(5.846935175196542, 22.70918264649648)
(7.307935699549983, 23.993893303167614)
(7.096135924975304, 24.57186963748485)
(9.32933794088265, 81.43237782492879)
(5.883964473245937, 35.434392540323906)
(5.563489733342735, 22.83474845400847)
(5.568631770522515, 18.445007769227757)
(6.749872334969524, 46.971682783340704)
(7.87152585561631, 63.84019986522051)
(4.740400496373423, 17.285554620923577)
(6.414356658474803, 14.519099967540235)
(2.6903482823009903, 13.028856697603496)
(8.678402392209112, 32.69313799960486)
(4.290231699039018, 18.742981199281864)
(4.488770340693511, 16.28704977830401)
(7.422610832921598, 37.48973780083237)
(2.7020972346630683, 13.805251121011889)
(5.29033929635065, 29.44649007185623)
(10.510561716821556, 75.42627916064974)
(4.208577844137574, 16.25959645005855)
(7.29604314878569, 26.014784022069747)
(5.570830626983202, 37.49599049001269)
(6.445311784218727, 48.31344591613444)
(5.815537859591965, 15.86928133430795)
(6.574310227460753, 52.126739491406234)
(7.080651042519

## Mean Differences

The following prices of White Tuna Packed in Oil or Water were collected by sampling major brands at a local super market:

In [43]:
oil = [1.27, 1.22, 1.19, 1.22]
water = [1.49, 1.29, 1.27, 1.35, 1.29, 1.00, 1.27, 1.28]

The sample means are:

In [44]:
np.mean(oil), np.mean(water)

(1.225, 1.28)

We wish to find a confidence interval for the difference between the population means for White Tuna Packed in Water or Oil. 

Note that we expect the statistic:

$$ Z = \frac{\bar{Y_1} - \bar{Y_2} - (\mu_1 - \mu_2) }{\sqrt{ \sigma_1^2/n_1 + \sigma_2^2/n_2 }} $$

to be a normally distributed. Assuming that $\sigma$ is equal for the two populations this would be:

$$ Z = \frac{\bar{Y_1} - \bar{Y_2} - (\mu_1 - \mu_2) }{ \sigma \sqrt{ 1/n_1 + 1/n_2} } $$

However note that for this problem we do not know $\sigma_1$ or $\sigma_2$, or $\sigma$ for that matter. The assumption that two very similar products have the same variance is a common one so we could proceed with that and use what is called a **pooled estimator** for the $\sigma$:

$$ S_p^2 = \frac{ (n_1 -1 ) S_1^2 + (n_2 - 1) S_2^2 }{n_1 + n_2 - 2} $$

To explain where this comes from:  

- note that we multiply the individual sample variances by their denominator.
- We compute the sum of the squares of the differences of each sample from the corresponding sample mean and add these
- then we divide by the degrees of freedom:  $n_1 + n_2 - 2$  The reason there are 2 less degrees of freedom is becuase we have two means.

- The other way to think about the pooled estimator is that it is the weighted average of the sample variances; weighted by the number of degrees of freedom.

Then we can use $S_p$ in our Student's T distirbution with $n_1 + n_2 - 2$ degrees of freedom:

$$ T = \frac{ \bar{Y_1} - \bar{Y_2} - (\mu_1 - \mu_2) }{ S_p \sqrt{1/n_1 + 1/n_2} }$$

We will get that:

$$ (\bar{Y}_1 - \bar{Y}_2) \pm t_{\alpha/2} S_p \sqrt{1/n_1 + 1/n_2} $$ 

gives the boundaries of our confidence interval where 

$$ P(-t_{\alpha/2} < T < t_{\alpha/2} ) = 1 - \alpha $$

In [45]:
# Doing the computation:

confidence = 0.99
alpha = 1 - confidence


# Compute the two sample means
Ybar1 = np.mean(water)
Ybar2 = np.mean(oil)

# Record the sample sizes
n1 = len(water)
n2 = len(oil)

# Compute the total degrees of freedom
dof = n1+n2-2

# Compute the two sample standard deviations
S1 = np.std(water, ddof = 1)
S2 = np.std(oil, ddof = 1)

# Compute the pooled sample standard deviation; 

Sp = np.sqrt( ((n1-1)*S1 + (n2-1)*S2)/dof )

# Compute the talpha

talpha = -t.ppf(alpha/2, dof)

# Give the confidence interval

( (Ybar1 - Ybar2) - talpha * Sp *np.sqrt(1/n1 + 1/n2), (Ybar1 - Ybar2) + talpha * Sp *np.sqrt(1/n1+1/n2) )


(-0.5724834871893034, 0.6824834871893033)